In [102]:
# Data manipulation
import pandas as pd
import numpy as np
# Evaluation of the model
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.size'] = 18
%matplotlib inline

MAX_EVALS = 15
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer



DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [103]:
import json
import shlex
import subprocess
import logging
from pprint import pprint
import re
import os


In [104]:
def runthebenchmark(hyperparameters):
    os.chdir('/home/meghaagr/project/progress/active/../')
    storeinfile(hyperparameters)
    out=subprocess.Popen(["python3","read_config.py"], shell=False, stdout=subprocess.PIPE)
    logging.basicConfig(level=logging.DEBUG)
    output=out.stdout.read().decode('utf-8')
    print(output)
    value = output.split(" ")[5]
    value = float(value)
    print(value)
    return float(1/value)
    #logging.basicConfig(filename='app.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s',level=logging.DEBUG)


In [105]:
def storeinfile(hyperparameters):
    data={"mpi": {"romio_ds_read": "enable", "romio_ds_write": "disable"}, "lfs": {"setstripe": {"size": "16777216", "count": 4}}}
    data["lfs"]["setstripe"]["size"] = int(hyperparameters["setstripe-size"])
    data["lfs"]["setstripe"]["count"] = int(hyperparameters["setstripe-count"])

    data["mpi"]["romio_ds_read"] = hyperparameters["romio_ds_read"]
    data["mpi"]["romio_ds_write"] = hyperparameters["romio_ds_write"]
    data["mpi"]["romio_cb_read"] = hyperparameters["romio_cb_read"]
    data["mpi"]["romio_cb_write"] = hyperparameters["romio_cb_write"]
    data["mpi"]["cb_buffer_size"] = str(int(hyperparameters["cb_buffer_size"]))
    with open("confex.json","w") as fp:
        json.dump(data,fp)
    print(data)


In [106]:
def objective(hyperparameters):
    global ITERATION
    ITERATION += 1
    start = timer()
    result = runthebenchmark(hyperparameters)
    run_time = timer() - start
    print({'loss': result, 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK})
    return {'loss': float(result), 'hyperparameters': hyperparameters, 'iteration': ITERATION, 'iteration_time': run_time, 'status': STATUS_OK}
    

In [107]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

space = {
    'romio_ds_read' : hp.choice('romio_ds_read',['enable','disable']),
    'romio_ds_write' : hp.choice('romio_ds_write',['enable','disable']),
    'romio_cb_read' : hp.choice('romio_cb_read',['enable','disable']),
    'romio_cb_write' : hp.choice('romio_cb_write',['enable','disable']),
    'cb_buffer_size' : 1048576*hp.qnormal('cb_buffer_size',512,128,4),
    'setstripe-size' : 65536*(hp.qloguniform('setstripe-size',0,3,1)),
    'setstripe-count' : hp.quniform('setstripe-count',0,20,1)
    
}

In [108]:
x = sample(space)
print(x)

{'cb_buffer_size': 591396864.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'disable', 'romio_ds_read': 'disable', 'romio_ds_write': 'disable', 'setstripe-count': 12.0, 'setstripe-size': 131072.0}


In [113]:
from hyperopt import tpe

# Create the algorithm
tpe_algorithm = tpe.suggest


from hyperopt import Trials

# Record results
trials = Trials()

from hyperopt import fmin


ITERATION = 0

best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials, max_evals = MAX_EVALS)

  0%|          | 0/15 [00:00<?, ?it/s, best loss: ?]

INFO:hyperopt.tpe:tpe_transform took 0.009608 seconds
INFO:hyperopt.tpe:TPE using 0 trials


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'romio_cb_read': 'enable', 'romio_cb_write': 'disable', 'cb_buffer_size': '398458880'}, 'lfs': {'setstripe': {'size': 131072, 'count': 6}}}
S3D-IO 200-400-400-2-4-4-1 851.60 3.81 4.59 431.69 19.07 45.24 0.16 0.17 131072 6 enable disable enable disable 398458880 

431.69                                              
{'loss': 0.0023164771016238503, 'hyperparameters': {'cb_buffer_size': 398458880.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'disable', 'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'setstripe-count': 6.0, 'setstripe-size': 131072.0}, 'iteration': 1, 'iteration_time': 55.62724397704005, 'status': 'ok'}
  7%|▋         | 1/15 [00:55<12:59, 55.65s/it, best loss: 0.0023164771016238503]

INFO:hyperopt.tpe:tpe_transform took 0.009512 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.002316


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'cb_buffer_size': '658505728'}, 'lfs': {'setstripe': {'size': 1048576, 'count': 16}}}
S3D-IO 200-400-400-2-4-4-1 473.80 3.81 8.25 1002.86 19.07 19.48 0.07 0.66 1048576 16 enable enable disable enable 658505728 

1002.86                                                                         
{'loss': 0.000997148156273059, 'hyperparameters': {'cb_buffer_size': 658505728.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 16.0, 'setstripe-size': 1048576.0}, 'iteration': 2, 'iteration_time': 33.32229568529874, 'status': 'ok'}
 13%|█▎        | 2/15 [01:29<10:36, 48.96s/it, best loss: 0.000997148156273059] 

...

{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'cb_buffer_size': '620756992'}, 'lfs': {'setstripe': {'size': 196608, 'count': 7}}}
S3D-IO 200-400-400-2-4-4-1 852.04 3.81 4.58 1164.05 19.07 16.78 0.19 0.33 196608 7 enable enable disable enable 620756992 

1164.05                                                                          
{'loss': 0.0008590696275933165, 'hyperparameters': {'cb_buffer_size': 620756992.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'enable', 'setstripe-count': 7.0, 'setstripe-size': 196608.0}, 'iteration': 13, 'iteration_time': 27.130868021398783, 'status': 'ok'}
 87%|████████▋ | 13/15 [07:51<01:13, 36.67s/it, best loss: 0.0008424670805988256]

INFO:hyperopt.tpe:tpe_transform took 0.010722 seconds
INFO:hyperopt.tpe:TPE using 13/13 trials with best loss 0.000842


{'mpi': {'romio_ds_read': 'disable', 'romio_ds_write': 'disable', 'romio_cb_read': 'enable', 'romio_cb_write': 'disable', 'cb_buffer_size': '499122176'}, 'lfs': {'setstripe': {'size': 262144, 'count': 16}}}
S3D-IO 200-400-400-2-4-4-1 853.53 3.81 4.58 980.43 19.07 19.92 0.13 0.16 262144 16 disable disable enable disable 499122176 

980.43                                                                           
{'loss': 0.0010199606295197006, 'hyperparameters': {'cb_buffer_size': 499122176.0, 'romio_cb_read': 'enable', 'romio_cb_write': 'disable', 'romio_ds_read': 'disable', 'romio_ds_write': 'disable', 'setstripe-count': 16.0, 'setstripe-size': 262144.0}, 'iteration': 14, 'iteration_time': 30.230907284654677, 'status': 'ok'}
 93%|█████████▎| 14/15 [08:21<00:34, 34.75s/it, best loss: 0.0008424670805988256]

INFO:hyperopt.tpe:tpe_transform took 0.009833 seconds
INFO:hyperopt.tpe:TPE using 14/14 trials with best loss 0.000842


{'mpi': {'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'cb_buffer_size': '557842432'}, 'lfs': {'setstripe': {'size': 1114112, 'count': 18}}}
S3D-IO 200-400-400-2-4-4-1 861.93 3.81 4.53 1110.49 19.07 17.59 0.16 0.34 1114112 18 enable disable disable enable 557842432 

1110.49                                                                          
{'loss': 0.0009005033813901971, 'hyperparameters': {'cb_buffer_size': 557842432.0, 'romio_cb_read': 'disable', 'romio_cb_write': 'enable', 'romio_ds_read': 'enable', 'romio_ds_write': 'disable', 'setstripe-count': 18.0, 'setstripe-size': 1114112.0}, 'iteration': 15, 'iteration_time': 27.959081003442407, 'status': 'ok'}
100%|██████████| 15/15 [08:49<00:00, 32.72s/it, best loss: 0.0008424670805988256]


In [114]:
print(best)
d=  best

{'cb_buffer_size': 576.0, 'romio_cb_read': 1, 'romio_cb_write': 1, 'romio_ds_read': 1, 'romio_ds_write': 0, 'setstripe-count': 9.0, 'setstripe-size': 1.0}
